In [1]:
## import libraries

import pandas as pd
import glob
# from tqdm import tqdm_notebook

from pathlib import Path
import sys

In [2]:
import sqlite3

In [3]:
%load_ext sql

In [4]:
%load_ext lab_black

# Link to Database

In [5]:
## import custom functions

# Create path to custom functions
funcs_path = Path.cwd().parents[1]
sys.path.insert(1, str(funcs_path))

from funcs import paths

In [6]:
## Get path to processed data folder

path_to_processed_data_folder = paths.get_path_to_folder("processed")

path_to_processed_data_folder

'C:\\Users\\RowanM\\Documents\\dublin_energy_masterplan\\Residential\\data\\inputs\\processed'

In [7]:
path_to_data_database = path_to_processed_data_folder + r"\data.db"

path_to_data_database

'C:\\Users\\RowanM\\Documents\\dublin_energy_masterplan\\Residential\\data\\inputs\\processed\\data.db'

In [8]:
db_filename = path_to_data_database

conn = sqlite3.connect(db_filename)  # or use :memory: to put it in RAM
cursor = conn.cursor()

In [9]:
%sql sqlite:///$path_to_data_database

'Connected: @C:\\Users\\RowanM\\Documents\\dublin_energy_masterplan\\Residential\\data\\inputs\\processed\\data.db'

In [10]:
%%sql 
SELECT * FROM electricity_data_raw
LIMIT 5

 * sqlite:///C:\Users\RowanM\Documents\dublin_energy_masterplan\Residential\data\inputs\processed\data.db
Done.


ID,Date_ID,Consumption,Day,Time_ID
1392,19503,0.14,195,3
1392,19504,0.138,195,4
1392,19505,0.14,195,5
1392,19506,0.145,195,6
1392,19507,0.145,195,7


# Find min and max Day to verify start/end date of trial vs pdf report of trial results

In [24]:
%%time
%%sql

SELECT MIN(Day) 
FROM electricity_data_raw

 * sqlite:///C:\Users\RowanM\Documents\dublin_energy_masterplan\Residential\data\inputs\processed\data.db
Done.
Wall time: 3min 14s


MIN(Day)
195


In [25]:
%%time
%%sql

SELECT MAX(Day) 
FROM electricity_data_raw

 * sqlite:///C:\Users\RowanM\Documents\dublin_energy_masterplan\Residential\data\inputs\processed\data.db
Done.
Wall time: 55.5 s


MAX(Day)
730


In [ ]:
%%time
%%sql

SELECT MIN(Time_ID) 
FROM electricity_data_raw

# Add column with dates

In [27]:
%%sql

ALTER TABLE electricity_data_raw
  ADD Date TEXT;

 * sqlite:///C:\Users\RowanM\Documents\dublin_energy_masterplan\Residential\data\inputs\processed\data.db
Done.


[]

In [28]:
%%time
%%sql

UPDATE electricity_data_raw
  SET Date = DATE('2009-01-01','+' || CAST(Day AS TEXT) || ' day')

 * sqlite:///C:\Users\RowanM\Documents\dublin_energy_masterplan\Residential\data\inputs\processed\data.db
157992996 rows affected.
Wall time: 11min 43s


[]

In [66]:
%%sql 
SELECT * FROM electricity_data_raw
LIMIT 5

 * sqlite:///C:\Users\RowanM\Documents\dublin_energy_masterplan\Residential\data\inputs\processed\data.db
Done.


ID,Date_ID,Consumption,Day,Time_ID,Year,Date
1392,19503,0.14,195,3,2009,2009-07-15
1392,19504,0.138,195,4,2009,2009-07-15
1392,19505,0.14,195,5,2009,2009-07-15
1392,19506,0.145,195,6,2009,2009-07-15
1392,19507,0.145,195,7,2009,2009-07-15


SOURCES: 
- Date SQLite function https://www.sqlitetutorial.net/sqlite-date-functions/sqlite-date-function/
- String concatanation https://www.sqlitetutorial.net/sqlite-string-functions/sqlite-concat/
- Integer to string https://stackoverflow.com/questions/12419693/convert-integer-to-text-in-sqlites-select-query
- Datetime type in sqlite https://stackoverflow.com/questions/17227110/how-do-datetime-values-work-in-sqlite

# Convert Time_ID to '00:00:00' format

In [68]:
%%sql

ALTER TABLE electricity_data_raw
  ADD Time TEXT;

 * sqlite:///C:\Users\RowanM\Documents\dublin_energy_masterplan\Residential\data\inputs\processed\data.db
Done.


[]

In [69]:
%%time
%%sql

UPDATE electricity_data_raw
    SET Time = TIME('00:00:00','+' || CAST( (Time_ID*24*60/48) AS text) || ' minutes')

 * sqlite:///C:\Users\RowanM\Documents\dublin_energy_masterplan\Residential\data\inputs\processed\data.db
157992996 rows affected.
Wall time: 14min 46s


[]

In [70]:
%%sql 
SELECT * FROM electricity_data_raw
LIMIT 5;

 * sqlite:///C:\Users\RowanM\Documents\dublin_energy_masterplan\Residential\data\inputs\processed\data.db
Done.


ID,Date_ID,Consumption,Day,Time_ID,Year,Date,Time
1392,19503,0.14,195,3,2009,2009-07-15,01:30:00
1392,19504,0.138,195,4,2009,2009-07-15,02:00:00
1392,19505,0.14,195,5,2009,2009-07-15,02:30:00
1392,19506,0.145,195,6,2009,2009-07-15,03:00:00
1392,19507,0.145,195,7,2009,2009-07-15,03:30:00


SOURCES: 
- Time SQLite function https://www.sqlitetutorial.net/sqlite-date-functions/sqlite-time-function/

# Roughwork

Add column for Year

Convert Day number to d-m-y

Convert Time_ID to 00:00:00 Format